In [1]:
from __future__ import absolute_import, division, print_function

import argparse
import logging
import os
import random
import glob

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from tensorboardX import SummaryWriter

from pytorch_transformers import (WEIGHTS_NAME, BertConfig,
                                  BertForQuestionAnswering, BertTokenizer,
                                  XLMConfig, XLMForQuestionAnswering,
                                  XLMTokenizer, XLNetConfig,
                                  XLNetForQuestionAnswering,
                                  XLNetTokenizer,
                                  DistilBertConfig, DistilBertForQuestionAnswering, DistilBertTokenizer)

from pytorch_transformers import AdamW, WarmupLinearSchedule

from utils_squad import (read_squad_examples, convert_examples_to_features,
                         RawResult, write_predictions,
                         RawResultExtended, write_predictions_extended)
from utils_squad_evaluate import EVAL_OPTS, main as evaluate_on_squad

#import tensorflow as tf

#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#config.gpu_options.visible_device_list = "0" #only the gpu 0 is allowed

# config.gpu_options.per_process_gpu_memory_fraction = 0.01

#set_session(tf.Session(config=config))

In [2]:
logger = logging.getLogger(__name__)

ALL_MODELS = sum((tuple(conf.pretrained_config_archive_map.keys()) \
                  for conf in (BertConfig, XLNetConfig, XLMConfig)), ())

In [3]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForQuestionAnswering, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForQuestionAnswering, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForQuestionAnswering, XLMTokenizer),
    'distilbert': (DistilBertConfig, DistilBertForQuestionAnswering, DistilBertTokenizer)
}

In [4]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [5]:
def train(args, train_dataset, model, tokenizer):
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps=args.warmup_steps, t_total=t_total)
    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank],
                                                          output_device=args.local_rank,
                                                          find_unused_parameters=True)

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info("  Total train batch size (w. parallel, distributed & accumulation) = %d",
                   args.train_batch_size * args.gradient_accumulation_steps * (torch.distributed.get_world_size() if args.local_rank != -1 else 1))
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0])
    set_seed(args)  # Added here for reproductibility (even between python 2 and 3)
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {'input_ids':       batch[0],
                      'attention_mask':  batch[1], 
                      'token_type_ids':  None if args.model_type == 'xlm' else batch[2],  
                      'start_positions': batch[3], 
                      'end_positions':   batch[4]}
            if args.model_type in ['xlnet', 'xlm']:
                inputs.update({'cls_index': batch[5],
                               'p_mask':       batch[6]})
            outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean() # mean() to average on multi-gpu parallel (not distributed) training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if args.local_rank == -1 and args.evaluate_during_training:  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar('eval_{}'.format(key), value, global_step)
                    tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar('loss', (tr_loss - logging_loss)/args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, 'checkpoint-{}'.format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    torch.save(args, os.path.join(output_dir, 'training_args.bin'))
                    logger.info("Saving model checkpoint to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

In [6]:
def evaluate(args, model, tokenizer, prefix=""):
    dataset, examples, features = load_and_cache_examples(args, tokenizer, evaluate=True, output_examples=True)

    if not os.path.exists(args.output_dir) and args.local_rank in [-1, 0]:
        os.makedirs(args.output_dir)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(dataset) if args.local_rank == -1 else DistributedSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    all_results = []
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)
        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': None if args.model_type == 'xlm' else batch[2]  # XLM don't use segment_ids
                      }
            example_indices = batch[3]
            if args.model_type in ['xlnet', 'xlm']:
                inputs.update({'cls_index': batch[4],
                               'p_mask':    batch[5]})
            outputs = model(**inputs)

        for i, example_index in enumerate(example_indices):
            eval_feature = features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
            if args.model_type in ['xlnet', 'xlm']:
                # XLNet uses a more complex post-processing procedure
                result = RawResultExtended(unique_id            = unique_id,
                                           start_top_log_probs  = to_list(outputs[0][i]),
                                           start_top_index      = to_list(outputs[1][i]),
                                           end_top_log_probs    = to_list(outputs[2][i]),
                                           end_top_index        = to_list(outputs[3][i]),
                                           cls_logits           = to_list(outputs[4][i]))
            else:
                result = RawResult(unique_id    = unique_id,
                                   start_logits = to_list(outputs[0][i]),
                                   end_logits   = to_list(outputs[1][i]))
            all_results.append(result)

    # Compute predictions
    output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(prefix))
    output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(prefix))
    if args.version_2_with_negative:
        output_null_log_odds_file = os.path.join(args.output_dir, "null_odds_{}.json".format(prefix))
    else:
        output_null_log_odds_file = None

    if args.model_type in ['xlnet', 'xlm']:
        # XLNet uses a more complex post-processing procedure
        write_predictions_extended(examples, features, all_results, args.n_best_size,
                        args.max_answer_length, output_prediction_file,
                        output_nbest_file, output_null_log_odds_file, args.predict_file,
                        model.config.start_n_top, model.config.end_n_top,
                        args.version_2_with_negative, tokenizer, args.verbose_logging)
    else:
        write_predictions(examples, features, all_results, args.n_best_size,
                        args.max_answer_length, args.do_lower_case, output_prediction_file,
                        output_nbest_file, output_null_log_odds_file, args.verbose_logging,
                        args.version_2_with_negative, args.null_score_diff_threshold)

    # Evaluate with the official SQuAD script
    evaluate_options = EVAL_OPTS(data_file=args.predict_file,
                                 pred_file=output_prediction_file,
                                 na_prob_file=output_null_log_odds_file)
    results = evaluate_on_squad(evaluate_options)
    return results

In [7]:
def load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False):
    if args.local_rank not in [-1, 0] and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Load data features from cache or dataset file
    input_file = args.predict_file if evaluate else args.train_file
    cached_features_file = 'cached_features_file'
    test_features_file = 'test_features_file'
    if os.path.exists(cached_features_file) and input_file == args.train_file:
        logger.info("Loading features from cached file %s", cached_features_file)
        examples = read_squad_examples(input_file=input_file,
                                            is_training=not evaluate,
                                            version_2_with_negative=args.version_2_with_negative)
        features = torch.load(cached_features_file)
    elif os.path.exists(test_features_file) and input_file == args.predict_file:
        logger.info("Loading features from cached file %s", cached_features_file)
        examples = read_squad_examples(input_file=input_file,
                                            is_training=not evaluate,
                                            version_2_with_negative=args.version_2_with_negative)
        features = torch.load(test_features_file)
        
    else:
        logger.info("Creating features from dataset file at %s", input_file)
        examples = read_squad_examples(input_file=input_file,
                                            is_training=not evaluate,
                                            version_2_with_negative=args.version_2_with_negative)
        features = convert_examples_to_features(examples=examples,
                                            tokenizer=tokenizer,
                                            max_seq_length=args.max_seq_length,
                                            doc_stride=args.doc_stride,
                                            max_query_length=args.max_query_length,
                                            is_training=not evaluate)
        # save train features into cached_features_file
        if args.local_rank in [-1, 0] and input_file == args.train_file:
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)
        # save test features into test_features_file
        if args.local_rank in [-1, 0] and input_file == args.predict_file:
            logger.info("Saving features into test file %s", test_features_file)
            torch.save(features, test_features_file)
    if args.local_rank == 0 and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
    all_p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)
    if evaluate:
        all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
        dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                all_example_index, all_cls_index, all_p_mask)
    else:
        all_start_positions = torch.tensor([f.start_position for f in features], dtype=torch.long)
        all_end_positions = torch.tensor([f.end_position for f in features], dtype=torch.long)
        dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                all_start_positions, all_end_positions,
                                all_cls_index, all_p_mask)

    if output_examples:
        return dataset, examples, features
    return dataset

In [8]:
def main():
    parser = argparse.ArgumentParser()

    ## Required parameters
    parser.add_argument("--train_file", default=None, type=str, required=True,
                        help="SQuAD json for training. E.g., train-v1.1.json")
    parser.add_argument("--predict_file", default=None, type=str, required=True,
                        help="SQuAD json for predictions. E.g., dev-v1.1.json or test-v1.1.json")
    parser.add_argument("--model_type", default=None, type=str, required=True,
                        help="Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys()))
    parser.add_argument("--model_name_or_path", default=None, type=str, required=True,
                        help="Path to pre-trained model or shortcut name selected in the list: " + ", ".join(ALL_MODELS))
    parser.add_argument("--output_dir", default=None, type=str, required=True,
                        help="The output directory where the model checkpoints and predictions will be written.")

    ## Other parameters
    parser.add_argument("--config_name", default="", type=str,
                        help="Pretrained config name or path if not the same as model_name")
    parser.add_argument("--tokenizer_name", default="", type=str,
                        help="Pretrained tokenizer name or path if not the same as model_name")
    parser.add_argument("--cache_dir", default="", type=str,
                        help="Where do you want to store the pre-trained models downloaded from s3")

    parser.add_argument('--version_2_with_negative', action='store_true',
                        help='If true, the SQuAD examples contain some that do not have an answer.')
    parser.add_argument('--null_score_diff_threshold', type=float, default=0.0,
                        help="If null_score - best_non_null is greater than the threshold predict null.")

    parser.add_argument("--max_seq_length", default=384, type=int,
                        help="The maximum total input sequence length after WordPiece tokenization. Sequences "
                             "longer than this will be truncated, and sequences shorter than this will be padded.")
    parser.add_argument("--doc_stride", default=128, type=int,
                        help="When splitting up a long document into chunks, how much stride to take between chunks.")
    parser.add_argument("--max_query_length", default=64, type=int,
                        help="The maximum number of tokens for the question. Questions longer than this will "
                             "be truncated to this length.")
    parser.add_argument("--do_train", action='store_true',
                        help="Whether to run training.")
    parser.add_argument("--do_eval", action='store_true',
                        help="Whether to run eval on the dev set.")
    parser.add_argument("--evaluate_during_training", action='store_true',
                        help="Rul evaluation during training at each logging step.")
    parser.add_argument("--do_lower_case", action='store_true',
                        help="Set this flag if you are using an uncased model.")

    parser.add_argument("--per_gpu_train_batch_size", default=8, type=int,
                        help="Batch size per GPU/CPU for training.")
    parser.add_argument("--per_gpu_eval_batch_size", default=8, type=int,
                        help="Batch size per GPU/CPU for evaluation.")
    parser.add_argument("--learning_rate", default=5e-5, type=float,
                        help="The initial learning rate for Adam.")
    parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
                        help="Number of updates steps to accumulate before performing a backward/update pass.")
    parser.add_argument("--weight_decay", default=0.0, type=float,
                        help="Weight deay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float,
                        help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float,
                        help="Max gradient norm.")
    parser.add_argument("--num_train_epochs", default=3.0, type=float,
                        help="Total number of training epochs to perform.")
    parser.add_argument("--max_steps", default=-1, type=int,
                        help="If > 0: set total number of training steps to perform. Override num_train_epochs.")
    parser.add_argument("--warmup_steps", default=0, type=int,
                        help="Linear warmup over warmup_steps.")
    parser.add_argument("--n_best_size", default=20, type=int,
                        help="The total number of n-best predictions to generate in the nbest_predictions.json output file.")
    parser.add_argument("--max_answer_length", default=30, type=int,
                        help="The maximum length of an answer that can be generated. This is needed because the start "
                             "and end predictions are not conditioned on one another.")
    parser.add_argument("--verbose_logging", action='store_true',
                        help="If true, all of the warnings related to data processing will be printed. "
                             "A number of warnings are expected for a normal SQuAD evaluation.")

    parser.add_argument('--logging_steps', type=int, default=50,
                        help="Log every X updates steps.")
    parser.add_argument('--save_steps', type=int, default=50,
                        help="Save checkpoint every X updates steps.")
    parser.add_argument("--eval_all_checkpoints", action='store_true',
                        help="Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number")
    parser.add_argument("--no_cuda", action='store_true',
                        help="Whether not to use CUDA when available")
    parser.add_argument('--overwrite_output_dir', action='store_true',
                        help="Overwrite the content of the output directory")
    parser.add_argument('--overwrite_cache', action='store_true',
                        help="Overwrite the cached training and evaluation sets")
    parser.add_argument('--seed', type=int, default=42,
                        help="random seed for initialization")

    parser.add_argument("--local_rank", type=int, default=-1,
                        help="local_rank for distributed training on gpus")
    parser.add_argument('--fp16', action='store_true',
                        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit")
    parser.add_argument('--fp16_opt_level', type=str, default='O1',
                        help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
                             "See details at https://nvidia.github.io/apex/amp.html")
    parser.add_argument('--server_ip', type=str, default='', help="Can be used for distant debugging.")
    parser.add_argument('--server_port', type=str, default='', help="Can be used for distant debugging.")
    #args = parser.parse_args()

    if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train and not args.overwrite_output_dir:
        raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args.output_dir))

    # Setup distant debugging if needed
    #if args.server_ip and args.server_port:
        # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
        #import ptvsd
        #print("Waiting for debugger attach")
        #ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
        #ptvsd.wait_for_attach()

    # Setup CUDA, GPU & distributed training
    #if args.local_rank == -1 or args.no_cuda:
        #device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        #args.n_gpu = torch.cuda.device_count()
    #else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        #torch.cuda.set_device(args.local_rank)
        #device = torch.device("cuda", args.local_rank)
        #torch.distributed.init_process_group(backend='nccl')
        #args.n_gpu = 1
    args.device = device

    # Setup logging
    #logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        #datefmt = '%m/%d/%Y %H:%M:%S',
                        #level = logging.INFO if args.local_rank in [-1, 0] else logging.WARN)
    #logger.warning("Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
                    #args.local_rank, device, args.n_gpu, bool(args.local_rank != -1), args.fp16)

    # Set seed
    #set_seed(args)

    # Load pretrained model and tokenizer
    #if args.local_rank not in [-1, 0]:
        #torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

    args.model_type = args.model_type.lower()
    config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
    config = config_class.from_pretrained(args.model_name)
    tokenizer = tokenizer_class.from_pretrained(args.model_name)
    model = model_class.from_pretrained(args.model_name, config=config)
    #if args.local_rank == 0:
        #torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

    model.to(args.device)

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False)
        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)


    # Save the trained model and the tokenizer
    if args.do_train: #and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
        # Create output directory if needed
        if not os.path.exists(args.output_dir) and args.local_rank in [-1, 0]:
            os.makedirs(args.output_dir)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))

        # Load a trained model and vocabulary that you have fine-tuned
        model = model_class.from_pretrained(args.output_dir)
        tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
        model.to(args.device)


    # Evaluation - we can ask to evaluate all the checkpoints (sub-directories) in a directory
    results = {}
    if args.do_eval: #and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + '/**/' + WEIGHTS_NAME, recursive=True)))
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce model loading logs

        logger.info("Evaluate the following checkpoints: %s", checkpoints)

        for checkpoint in checkpoints:
            # Reload the model
            global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
            model = model_class.from_pretrained(checkpoint)
            model.to(args.device)

            # Evaluate
            result = evaluate(args, model, tokenizer, prefix=global_step)

            result = dict((k + ('_{}'.format(global_step) if global_step else ''), v) for k, v in result.items())
            results.update(result)

    logger.info("Results: {}".format(results))

    return results

In [9]:
class struct():
    def __init__(self):
        self.train_file = 'train-v2.0.json'
        self.predict_file = 'dev-v2.0.json'
        # exam the new dataset
        #self.predict_file = 'test.json'
        self.model_type = 'bert'
        self.model_name = 'bert-large-uncased-whole-word-masking'
        self.task_name = 'MRPC'
        self.do_train = True
        self.do_eval = True
        self.do_lower_case = True
        self.data_dir = 'GLUE_DIR/MRPC/'
        self.max_seq_length = 384
        self.per_gpu_eval_batch_size = 12
        self.per_gpu_train_batch_size = 12
        # the default is 2, we make it quicker
        self.num_train_epochs = 2.0
        self.learning_rate = 3e-5
        self.output_dir = 'tmp/mrpc_output/'
        self.overwrite_output_dir = True
        self.overwrite_cache = True   
        self.local_rank = -1
        self.version_2_with_negative = True
        self.doc_stride = 128
        self.max_query_length = 64
        self.n_gpu = 1
        self.max_steps = -1
        self.gradient_accumulation_steps = 1
        self.weight_decay = 0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1
        self.warmup_steps = 0
        self.n_best_size = 20
        self.max_answer_length = 30
        self.verbose_logging = True
        self.logging_steps = 50
        self.save_steps = 50000
        self.fp16 = False
        self.fp16_opt_level = 'O1'
        self.seed = 42
        self.no_cuda = False
        self.evaluate_during_training = False
        

In [10]:
args = struct()

In [11]:
args.predict_file

'dev-v2.0.json'

In [12]:
input_file = args.train_file
input_file

'train-v2.0.json'

In [13]:
read_squad_examples(input_file=input_file,
                            is_training=not evaluate,
                            version_2_with_negative=False)

[qas_id: 56be85543aeaaa14008c9063, question_text: When did Beyonce start becoming popular?, doc_tokens: [Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".],
 qas_id: 56be85543aeaaa14008c9065, question_text: What areas did Beyonce compete in when she was growing up?, doc_tokens: [Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4

In [13]:
# set up device
if args.local_rank == -1 or args.no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = torch.cuda.device_count()
else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    torch.distributed.init_process_group(backend='nccl')
    args.n_gpu = 1
args.device = device

In [15]:
# prepare model and dataset
args.model_type = args.model_type.lower()
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
config = config_class.from_pretrained(args.model_name)
tokenizer = tokenizer_class.from_pretrained(args.model_name)
model = model_class.from_pretrained(args.model_name, config=config)

model.to(args.device)
logger.info("Training/evaluation parameters %s", args)
    
train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False)

In [14]:
args.fp16

False

In [17]:
# start training
global_step, tr_loss = train(args, train_dataset, model, tokenizer)

Iteration:   0%|          | 0/5498 [00:00<?, ?it/s]/home/xwan6/.local/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

Iteration:   4%|▍         | 240/5498 [04:17<1:32:08,  1.05s/it]


Iteration:   9%|▉         | 484/5498 [08:34<1:27:54,  1.05s/it]


Iteration:  13%|█▎        | 728/5498 [12:52<1:23:41,  1.05s/it]


Iteration:  18%|█▊        | 972/5498 [17:10<1:19:18,  1.05s/it]


Iteration:  22%|██▏       | 1212/5498 [21:23<1:15:17,  1.05s/it]


...

Iteration:  85%|████████▌ | 4695/5498 [1:22:35<14:05,  1.05s/it]


Iteration:  90%|████████▉ | 4935/5498 [1:26:48<09:52,  1.05s/it]


Iteration:  94%|█████████▍| 5175/5498 [1:31:02<05:40,  1.05s/it]


Iteration:  98%|█████████▊| 5415/5498 [1:35:15<01:27,  1.05s/it]


Epoch: 100%|██████████| 2/2 [3:13:29<00:00, 5805.66s/it]  3it/s]


In [18]:
logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

In [21]:
# save the trained model and the tokenizer
#logger.info("Saving model checkpoint to %s", args.output_dir)
# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
#model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
#model_to_save.save_pretrained(args.output_dir)
#tokenizer.save_pretrained(args.output_dir)

('tmp/mrpc_output/vocab.txt',
 'tmp/mrpc_output/special_tokens_map.json',
 'tmp/mrpc_output/added_tokens.json')

In [151]:
 # Good practice: save your training arguments together with the trained model
#torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))

In [14]:
# Load a trained model and vocabulary that you have fine-tuned
args.model_type = args.model_type.lower()
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
config = config_class.from_pretrained(args.model_name)
model = model_class.from_pretrained(args.output_dir)
tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
model.to(args.device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

In [66]:
# Evaluation - we can ask to evaluate all the checkpoints (sub-directories) in a directory (too lone)
#checkpoints = [args.output_dir]
#checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + '/**/' + WEIGHTS_NAME, recursive=True)))
#logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce model loading logs
#logger.info("Evaluate the following checkpoints: %s", checkpoints)

In [22]:
# evaluate the final results
dataset, examples, features = load_and_cache_examples(args, tokenizer, evaluate=True, output_examples=True)
args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
# Note that DistributedSampler samples randomly
eval_sampler = SequentialSampler(dataset) if args.local_rank == -1 else DistributedSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

In [23]:
# Eval!
prefix=46530
logger.info("***** Running evaluation {} *****".format(prefix))
logger.info("  Num examples = %d", len(dataset))
logger.info("  Batch size = %d", args.eval_batch_size)
all_results = []
for batch in tqdm(eval_dataloader, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(args.device) for t in batch)
    with torch.no_grad():
        inputs = {'input_ids':      batch[0],
                    'attention_mask': batch[1],
                    'token_type_ids': None if args.model_type == 'xlm' else batch[2]  # XLM don't use segment_ids
                    }
        example_indices = batch[3]
        if args.model_type in ['xlnet', 'xlm']:
            inputs.update({'cls_index': batch[4],
                            'p_mask':    batch[5]})
        outputs = model(**inputs)

    for i, example_index in enumerate(example_indices):
        eval_feature = features[example_index.item()]
        unique_id = int(eval_feature.unique_id)
        if args.model_type in ['xlnet', 'xlm']:
            # XLNet uses a more complex post-processing procedure
            result = RawResultExtended(unique_id            = unique_id,
                                        start_top_log_probs  = to_list(outputs[0][i]),
                                        start_top_index      = to_list(outputs[1][i]),
                                        end_top_log_probs    = to_list(outputs[2][i]),
                                        end_top_index        = to_list(outputs[3][i]),
                                        cls_logits           = to_list(outputs[4][i]))
        else:
            result = RawResult(unique_id    = unique_id,
                                start_logits = to_list(outputs[0][i]),
                                end_logits   = to_list(outputs[1][i]))
        all_results.append(result)
#result = evaluate(args, model, tokenizer, prefix=global_step)

Evaluating: 100%|██████████| 510/510 [05:26<00:00,  1.75it/s]


In [24]:
# Compute predictions
output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(prefix))
output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(prefix))
# assume some question does not have an answer
if args.version_2_with_negative:
    output_null_log_odds_file = os.path.join(args.output_dir, "null_odds_{}.json".format(prefix))

In [25]:
args.null_score_diff_threshold = 0
args.version_2_with_negative = True
if args.model_type in ['xlnet', 'xlm']:
    # XLNet uses a more complex post-processing procedure
    write_predictions_extended(examples, features, all_results, args.n_best_size,
                    args.max_answer_length, output_prediction_file,
                    output_nbest_file, output_null_log_odds_file, args.predict_file,
                    model.config.start_n_top, model.config.end_n_top,
                    args.version_2_with_negative, tokenizer, args.verbose_logging)
else:
    write_predictions(examples, features, all_results, args.n_best_size,
                    args.max_answer_length, args.do_lower_case, output_prediction_file,
                    output_nbest_file, output_null_log_odds_file, args.verbose_logging,
                    args.version_2_with_negative, args.null_score_diff_threshold)

In [26]:
# Evaluate with the official SQuAD script
evaluate_options = EVAL_OPTS(data_file=args.predict_file,
                            pred_file=output_prediction_file,
                            na_prob_file=output_null_log_odds_file)
results = evaluate_on_squad(evaluate_options)

{
  "exact": 82.12751621325697,
  "f1": 85.07844733920284,
  "total": 11873,
  "HasAns_exact": 80.1450742240216,
  "HasAns_f1": 86.05539899769825,
  "HasAns_total": 5928,
  "NoAns_exact": 84.10428931875526,
  "NoAns_f1": 84.10428931875526,
  "NoAns_total": 5945,
  "best_exact": 82.89396108818327,
  "best_exact_thresh": -2.081042766571045,
  "best_f1": 85.69798230133034,
  "best_f1_thresh": -2.081042766571045
}


In [219]:
# manually compare the true and predicted answer
import json
output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(46530))
filename1 = output_prediction_file
filename2 = 'dev-v2.0.json'
with open(filename1, 'r') as f:
    data_predict = json.load(f)
with open(filename2, 'r') as f:
    data_true = json.load(f)    

In [221]:
data_true['data'][0]

{'title': 'Normans',
 'paragraphs': [{'qas': [{'question': 'In what country is Normandy located?',
     'id': '56ddde6b9a695914005b9628',
     'answers': [{'text': 'France', 'answer_start': 159},
      {'text': 'France', 'answer_start': 159},
      {'text': 'France', 'answer_start': 159},
      {'text': 'France', 'answer_start': 159}],
     'is_impossible': False},
    {'question': 'When were the Normans in Normandy?',
     'id': '56ddde6b9a695914005b9629',
     'answers': [{'text': '10th and 11th centuries', 'answer_start': 94},
      {'text': 'in the 10th and 11th centuries', 'answer_start': 87},
      {'text': '10th and 11th centuries', 'answer_start': 94},
      {'text': '10th and 11th centuries', 'answer_start': 94}],
     'is_impossible': False},
    {'question': 'From which countries did the Norse originate?',
     'id': '56ddde6b9a695914005b962a',
     'answers': [{'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
      {'text': 'Denmark, Iceland and Norway', 'answer

In [149]:
data_predict

{'56ddde6b9a695914005b9628': 'France',
 '56ddde6b9a695914005b9629': '10th and 11th centuries',
 '56ddde6b9a695914005b962a': 'Denmark, Iceland and Norway',
 '56ddde6b9a695914005b962b': 'Rollo',
 '56ddde6b9a695914005b962c': '10th',
 '5ad39d53604f3c001a3fe8d1': '',
 '5ad39d53604f3c001a3fe8d2': '',
 '5ad39d53604f3c001a3fe8d3': '',
 '5ad39d53604f3c001a3fe8d4': '',
 '56dddf4066d3e219004dad5f': 'William the Conqueror',
 '56dddf4066d3e219004dad60': 'Richard I of Normandy',
 '56dddf4066d3e219004dad61': 'Christian',
 '5ad3a266604f3c001a3fea27': '',
 '5ad3a266604f3c001a3fea28': 'The Normans',
 '5ad3a266604f3c001a3fea29': '',
 '5ad3a266604f3c001a3fea2a': 'Richard I of Normandy',
 '5ad3a266604f3c001a3fea2b': '',
 '56dde0379a695914005b9636': 'Norseman, Viking',
 '56dde0379a695914005b9637': '9th century',
 '5ad3ab70604f3c001a3feb89': '',
 '5ad3ab70604f3c001a3feb8a': '',
 '56dde0ba66d3e219004dad75': '911',
 '56dde0ba66d3e219004dad76': '',
 '56dde0ba66d3e219004dad77': 'the river Epte and the Atlantic c

In [192]:
# learning the input and output data format
import json
filename1 = args.train_file
filename2 = args.predict_file
with open(filename1, 'r') as f:
    data_train = json.load(f)
with open(filename2, 'r') as f:
    data_true = json.load(f)    

In [21]:
data_true.keys()

dict_keys(['version', 'data'])

In [28]:
data_true['data'][0]['paragraphs'][0]['qas']

[{'question': 'In what country is Normandy located?',
  'id': '56ddde6b9a695914005b9628',
  'answers': [{'text': 'France', 'answer_start': 159},
   {'text': 'France', 'answer_start': 159},
   {'text': 'France', 'answer_start': 159},
   {'text': 'France', 'answer_start': 159}],
  'is_impossible': False},
 {'question': 'When were the Normans in Normandy?',
  'id': '56ddde6b9a695914005b9629',
  'answers': [{'text': '10th and 11th centuries', 'answer_start': 94},
   {'text': 'in the 10th and 11th centuries', 'answer_start': 87},
   {'text': '10th and 11th centuries', 'answer_start': 94},
   {'text': '10th and 11th centuries', 'answer_start': 94}],
  'is_impossible': False},
 {'question': 'From which countries did the Norse originate?',
  'id': '56ddde6b9a695914005b962a',
  'answers': [{'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
   {'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
   {'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
   {'text': 'Den

In [196]:
data_train['data'][0]['paragraphs'][0]

{'qas': [{'question': 'When did Beyonce start becoming popular?',
   'id': '56be85543aeaaa14008c9063',
   'answers': [{'text': 'in the late 1990s', 'answer_start': 269}],
   'is_impossible': False},
  {'question': 'What areas did Beyonce compete in when she was growing up?',
   'id': '56be85543aeaaa14008c9065',
   'answers': [{'text': 'singing and dancing', 'answer_start': 207}],
   'is_impossible': False},
  {'question': "When did Beyonce leave Destiny's Child and become a solo singer?",
   'id': '56be85543aeaaa14008c9066',
   'answers': [{'text': '2003', 'answer_start': 526}],
   'is_impossible': False},
  {'question': 'In what city and state did Beyonce  grow up? ',
   'id': '56bf6b0f3aeaaa14008c9601',
   'answers': [{'text': 'Houston, Texas', 'answer_start': 166}],
   'is_impossible': False},
  {'question': 'In which decade did Beyonce become famous?',
   'id': '56bf6b0f3aeaaa14008c9602',
   'answers': [{'text': 'late 1990s', 'answer_start': 276}],
   'is_impossible': False},
  {'q

In [16]:
# create similar input data
test_grandson = {}
test_grandson['qas'] = [{'question':'What happened?',
                         'id': '1',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False},
                        {'question':'Where is the delay?',
                         'id': '2',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False},
                       {'question':'Where is the construction?',
                         'id': '3',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False},
                       {'question':'Where is the incident?',
                         'id': '4',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False},
                       {'question':'When is the delay?',
                         'id': '5',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False},
                       {'question':'When is the incident?',
                         'id': '6',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False},
                       {'question':'When is the construction?',
                         'id': '7',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False}]
test_grandson['context'] = 'An accident on 73 SB approaching Fellowship Rd is causing a delay between 295 and the NJ Turnpike.'

In [17]:
test_child = {}
test_child['title'] =  'Traffic'
test_child['paragraphs'] = [test_grandson]

In [18]:
test = {}
test['version'] = 'test'
test['data'] = [test_child]

In [19]:
test

{'version': 'test',
 'data': [{'title': 'Traffic',
   'paragraphs': [{'qas': [{'question': 'What happened?',
       'id': '1',
       'answers': [{'text': 'food', 'answer_start': 0}],
       'is_impossible': False},
      {'question': 'Where is the delay?',
       'id': '2',
       'answers': [{'text': 'food', 'answer_start': 0}],
       'is_impossible': False},
      {'question': 'Where is the construction?',
       'id': '3',
       'answers': [{'text': 'food', 'answer_start': 0}],
       'is_impossible': False},
      {'question': 'Where is the incident?',
       'id': '4',
       'answers': [{'text': 'food', 'answer_start': 0}],
       'is_impossible': False},
      {'question': 'When is the delay?',
       'id': '5',
       'answers': [{'text': 'food', 'answer_start': 0}],
       'is_impossible': False},
      {'question': 'When is the incident?',
       'id': '6',
       'answers': [{'text': 'food', 'answer_start': 0}],
       'is_impossible': False},
      {'question': 'When is 

In [20]:
# save test to json
import json
with open('test.json', 'w') as outfile:
    json.dump(test, outfile)

In [17]:
# we need to change the datasets
def load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False):
    if args.local_rank not in [-1, 0] and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Load data features from cache or dataset file
    input_file = args.predict_file if evaluate else args.train_file
    cached_features_file = 'cached_features_file'
    test_features_file = 'test_features_file'
    
    logger.info("Creating features from dataset file at %s", input_file)
    examples = read_squad_examples(input_file=input_file,
                                        is_training=not evaluate,
                                        version_2_with_negative=args.version_2_with_negative)
    features = convert_examples_to_features(examples=examples,
                                        tokenizer=tokenizer,
                                        max_seq_length=args.max_seq_length,
                                        doc_stride=args.doc_stride,
                                        max_query_length=args.max_query_length,
                                            is_training=not evaluate)
    
    if args.local_rank == 0 and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
    all_p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)
    if evaluate:
        all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
        dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                all_example_index, all_cls_index, all_p_mask)
    else:
        all_start_positions = torch.tensor([f.start_position for f in features], dtype=torch.long)
        all_end_positions = torch.tensor([f.end_position for f in features], dtype=torch.long)
        dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                all_start_positions, all_end_positions,
                                all_cls_index, all_p_mask)

    if output_examples:
        return dataset, examples, features
    return dataset

In [22]:
# evaluate the QA for the new dataset
# evaluate the final results
args.predict_file = 'test.json'
dataset, examples, features = load_and_cache_examples(args, tokenizer, evaluate=True, output_examples=True)
args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
# Note that DistributedSampler samples randomly
eval_sampler = SequentialSampler(dataset) if args.local_rank == -1 else DistributedSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

In [23]:
prefix=1
args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
logger.info("***** Running evaluation {} *****".format(prefix))
logger.info("  Num examples = %d", len(dataset))
logger.info("  Batch size = %d", args.eval_batch_size)
all_results = []
for batch in tqdm(eval_dataloader, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(args.device) for t in batch)
    with torch.no_grad():
        inputs = {'input_ids':      batch[0],
                    'attention_mask': batch[1],
                    'token_type_ids': None if args.model_type == 'xlm' else batch[2]  # XLM don't use segment_ids
                    }
        example_indices = batch[3]
        if args.model_type in ['xlnet', 'xlm']:
            inputs.update({'cls_index': batch[4],
                            'p_mask':    batch[5]})
        outputs = model(**inputs)

    for i, example_index in enumerate(example_indices):
        eval_feature = features[example_index.item()]
        unique_id = int(eval_feature.unique_id)
        if args.model_type in ['xlnet', 'xlm']:
            # XLNet uses a more complex post-processing procedure
            result = RawResultExtended(unique_id            = unique_id,
                                        start_top_log_probs  = to_list(outputs[0][i]),
                                        start_top_index      = to_list(outputs[1][i]),
                                        end_top_log_probs    = to_list(outputs[2][i]),
                                        end_top_index        = to_list(outputs[3][i]),
                                        cls_logits           = to_list(outputs[4][i]))
        else:
            result = RawResult(unique_id    = unique_id,
                                start_logits = to_list(outputs[0][i]),
                                end_logits   = to_list(outputs[1][i]))
        all_results.append(result)

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  4.67it/s]


In [43]:
# Compute predictions
output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(prefix))
output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(prefix))
# assume some question does not have an answer
if args.version_2_with_negative:
    output_null_log_odds_file = os.path.join(args.output_dir, "null_odds_{}.json".format(prefix))

In [44]:
args.null_score_diff_threshold = 0
args.version_2_with_negative = True
if args.model_type in ['xlnet', 'xlm']:
    # XLNet uses a more complex post-processing procedure
    write_predictions_extended(examples, features, all_results, args.n_best_size,
                    args.max_answer_length, output_prediction_file,
                    output_nbest_file, output_null_log_odds_file, args.predict_file,
                    model.config.start_n_top, model.config.end_n_top,
                    args.version_2_with_negative, tokenizer, args.verbose_logging)
else:
    write_predictions(examples, features, all_results, args.n_best_size,
                    args.max_answer_length, args.do_lower_case, output_prediction_file,
                    output_nbest_file, output_null_log_odds_file, args.verbose_logging,
                    args.version_2_with_negative, args.null_score_diff_threshold)

In [45]:
# read the answer
import json
filename1 = output_prediction_file
with open(filename1, 'r') as f:
    answer = json.load(f)

In [46]:
answer

{'1': 'An accident on 73 SB approaching Fellowship Rd is causing a delay between 295 and the NJ Turnpike',
 '2': 'between 295 and the NJ Turnpike',
 '3': '',
 '4': 'Fellowship Rd',
 '5': 'between 295 and the NJ Turnpike',
 '6': 'An accident on 73 SB approaching Fellowship Rd',
 '7': ''}

In [18]:
import pandas as pd
data = pd.read_csv('local.csv')

In [19]:
# collect all the information
# collect all answers from tweets
import time
import json
answer_collect = {}
for key in range(len(data)):
#for key in range(3):
    context = data.iloc[key]['text']
    context = context.replace('#','')
    test_grandson = {}
    test_grandson['qas'] = [{'question':'What happened?',
                         'id': '1',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False},
                        {'question':'Where is the delay?',
                         'id': '2',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False}, 
                       {'question':'Where is the construction?',
                         'id': '3',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False},
                       {'question':'Where is the incident?',
                         'id': '4',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False},
                       {'question':'Where is the event?',
                         'id': '5',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False},
                       {'question':'Where is the closure?',
                         'id': '6',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False},
                       {'question':'Cleared or Updated?',
                         'id': '7',
                          'answers': [{'text':'food','answer_start': 0}],
                          'is_impossible': False}]
    test_grandson['context'] = context

    test_child = {}
    test_child['title'] =  'Thai'
    test_child['paragraphs'] = [test_grandson]

    test = {}
    test['version'] = 'test'
    test['data'] = [test_child]
    with open('test.json', 'w') as outfile:
        json.dump(test, outfile)
    #time.sleep(1)
    # evaluate the QA for the new dataset
    # evaluate the final results
    args.predict_file = 'test.json'
    dataset, examples, features = load_and_cache_examples(args, tokenizer, evaluate=True, output_examples=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(dataset) if args.local_rank == -1 else DistributedSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)
    
    prefix=1
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    all_results = []
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)
        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                        'attention_mask': batch[1],
                        'token_type_ids': None if args.model_type == 'xlm' else batch[2]  # XLM don't use segment_ids
                        }
            example_indices = batch[3]
            if args.model_type in ['xlnet', 'xlm']:
                inputs.update({'cls_index': batch[4],
                                'p_mask':    batch[5]})
            outputs = model(**inputs)
    
        for i, example_index in enumerate(example_indices):
            eval_feature = features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
            if args.model_type in ['xlnet', 'xlm']:
                # XLNet uses a more complex post-processing procedure
                result = RawResultExtended(unique_id            = unique_id,
                                            start_top_log_probs  = to_list(outputs[0][i]),
                                            start_top_index      = to_list(outputs[1][i]),
                                            end_top_log_probs    = to_list(outputs[2][i]),
                                            end_top_index        = to_list(outputs[3][i]),
                                            cls_logits           = to_list(outputs[4][i]))
            else:
                result = RawResult(unique_id    = unique_id,
                                    start_logits = to_list(outputs[0][i]),
                                    end_logits   = to_list(outputs[1][i]))
            all_results.append(result)
            
    # Compute predictions
    output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(prefix))
    output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(prefix))
    # assume some question does not have an answer
    if args.version_2_with_negative:
        output_null_log_odds_file = os.path.join(args.output_dir, "null_odds_{}.json".format(prefix))
        
    args.null_score_diff_threshold = 0
    args.version_2_with_negative = True
    if args.model_type in ['xlnet', 'xlm']:
        # XLNet uses a more complex post-processing procedure
        write_predictions_extended(examples, features, all_results, args.n_best_size,
                        args.max_answer_length, output_prediction_file,
                        output_nbest_file, output_null_log_odds_file, args.predict_file,
                        model.config.start_n_top, model.config.end_n_top,
                        args.version_2_with_negative, tokenizer, args.verbose_logging)
    else:
        write_predictions(examples, features, all_results, args.n_best_size,
                        args.max_answer_length, args.do_lower_case, output_prediction_file,
                        output_nbest_file, output_null_log_odds_file, args.verbose_logging,
                        args.version_2_with_negative, args.null_score_diff_threshold)
        
    filename1 = output_prediction_file
    with open(filename1, 'r') as f:
        answer = json.load(f)    
    #time.sleep(1)    
    answer_collect[key] = answer

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.64it/s]


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.10it/s]


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.18it/s]


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.28it/s]


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.15it/s]


...

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.27it/s]


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.14it/s]


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.23it/s]


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.22it/s]


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.21it/s]


In [20]:
answer_collect[2]

{'1': 'Construction on GeorgeWashingtonBridge WB at Lower Trans Manhattan Expressway',
 '2': '',
 '3': 'Lower Trans Manhattan Expressway',
 '4': 'Lower Trans Manhattan Expressway',
 '5': 'Lower Trans Manhattan Expressway',
 '6': '',
 '7': ''}

In [22]:
len(answer_collect)

1867

In [23]:
df = pd.DataFrame(columns=['What happened?','Where is the delay?','Where is the construction?','Where is the incident?',
                           'Where is the event?','Where is the closure?','Cleared or Updated?'])

In [24]:
for i in range(len(answer_collect)):
    df.at[i,'What happened?'] = answer_collect[i]['1']
    df.at[i,'Where is the delay?'] = answer_collect[i]['2']
    df.at[i,'Where is the construction?'] = answer_collect[i]['3']
    df.at[i,'Where is the incident?'] = answer_collect[i]['4']
    df.at[i,'Where is the event?'] = answer_collect[i]['5']
    df.at[i,'Where is the closure?'] = answer_collect[i]['6']
    df.at[i,'Cleared or Updated?'] = answer_collect[i]['7']

In [25]:
df.to_csv('df.csv')